# Automatic generation of Notebook using PyCropML
    This notebook implements a crop model.

### Model Updateleafflag

In [ ]:
MODULE list_sub
    IMPLICIT NONE
    TYPE container
        CLASS(*), ALLOCATABLE :: item
        CLASS(*), ALLOCATABLE :: items(:)
    END TYPE

    interface Add
        module procedure AddToListFloat
        module procedure AddToListInt
        module procedure AddToListChar
        module procedure AddToListArray
    end interface
CONTAINS

    FUNCTION indice(vectorElem, elem)
        CHARACTER(LEN=*), DIMENSION(:):: vectorElem
        INTEGER::iterator, indice
        CHARACTER(LEN=*):: elem
        DO iterator= 1, SIZE(vectorElem)
            IF(vectorElem(iterator)==elem) THEN
                indice = iterator
            END IF
        END DO
        RETURN
    END FUNCTION indice


    SUBROUTINE AddToListFloat(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize
        REAL, INTENT(IN) :: element
        REAL, DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        REAL, DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            ALLOCATE(list(1))
            list(1) = element
        END IF
    END SUBROUTINE AddToListFloat

    SUBROUTINE AddToListInt(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize
        INTEGER, INTENT(IN) :: element
        INTEGER, DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        INTEGER, DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            ALLOCATE(list(1))
            list(1) = element
        END IF
    END SUBROUTINE AddToListInt

    SUBROUTINE AddToListChar(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize, l
        CHARACTER(LEN=*), INTENT(IN) :: element
        CHARACTER(LEN=*), DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        CHARACTER(LEN=65), DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            l=1
            ALLOCATE(list(l))
            list(l) = element
        END IF
    END SUBROUTINE AddToListChar


    SUBROUTINE AddToListArray(a, e)
        TYPE(container),ALLOCATABLE,INTENT(INOUT) :: a(:)
        CLASS(*),INTENT(IN), allocatable :: e(:)
        TYPE(container),ALLOCATABLE :: tmp(:)

        IF (.NOT.ALLOCATED(a)) THEN
            ALLOCATE(a(1))
            ALLOCATE(a(1)%items(SIZE(e)), source = e)
        ELSE
            CALL MOVE_ALLOC(a,tmp)
            ALLOCATE(a(SIZE(tmp)+1))
            a(1:SIZE(tmp)) = tmp
            ALLOCATE(a(SIZE(tmp)+1)%items(SIZE(e)), source = e)
        END IF
    END SUBROUTINE AddToListArray

END MODULE list_sub
MODULE Updateleafflagmod
    USE list_sub
    IMPLICIT NONE
CONTAINS
    SUBROUTINE model_updateleafflag(cumulTT, &
        leafNumber, &
        calendarMoments, &
        calendarDates, &
        calendarCumuls, &
        currentdate, &
        finalLeafNumber, &
        hasFlagLeafLiguleAppeared, &
        phase)
        REAL, INTENT(IN) :: cumulTT
        REAL, INTENT(IN) :: leafNumber
        CHARACTER(65), ALLOCATABLE , DIMENSION(:), INTENT(INOUT) ::  &
                calendarMoments
        CHARACTER(65), ALLOCATABLE , DIMENSION(:), INTENT(INOUT) ::  &
                calendarDates
        REAL, ALLOCATABLE , DIMENSION(:), INTENT(INOUT) :: calendarCumuls
        CHARACTER(65), INTENT(IN) :: currentdate
        REAL, INTENT(IN) :: finalLeafNumber
        INTEGER, INTENT(INOUT) :: hasFlagLeafLiguleAppeared
        REAL, INTENT(IN) :: phase
        !- Description:
    !            - Model Name: UpdateLeafFlag Model
    !            - Author: Pierre MARTRE
    !            - Reference: Modeling development phase in the 
    !                Wheat Simulation Model SiriusQuality.
    !                See documentation at http://www1.clermont.inra.fr/siriusquality/?page_id=427
    !            - Institution: INRA Montpellier
    !            - Abstract: tells if flag leaf has appeared and update the calendar if so
    !    	
        !- inputs:
    !            - name: cumulTT
    !                          - description : cumul thermal times at current date
    !                          - variablecategory : auxiliary
    !                          - datatype : DOUBLE
    !                          - min : -200
    !                          - max : 10000
    !                          - default : 741.510096671757
    !                          - unit : °C d
    !                          - uri : some url
    !                          - inputtype : variable
    !            - name: leafNumber
    !                          - description : Actual number of phytomers
    !                          - variablecategory : state
    !                          - datatype : DOUBLE
    !                          - min : 0
    !                          - max : 25
    !                          - default : 8.919453833361189
    !                          - unit : leaf
    !                          - uri : some url
    !                          - inputtype : variable
    !            - name: calendarMoments
    !                          - description : List containing apparition of each stage
    !                          - variablecategory : state
    !                          - datatype : STRINGLIST
    !                          - default : ['Sowing']
    !                          - unit : 
    !                          - inputtype : variable
    !            - name: calendarDates
    !                          - description : List containing  the dates of the wheat developmental phases
    !                          - variablecategory : state
    !                          - datatype : DATELIST
    !                          - default : ['21/3/2007']
    !                          - unit : 
    !                          - inputtype : variable
    !            - name: calendarCumuls
    !                          - description : list containing for each stage occured its cumulated thermal times
    !                          - variablecategory : state
    !                          - datatype : DOUBLELIST
    !                          - default : [0.0]
    !                          - unit : °C d
    !                          - inputtype : variable
    !            - name: currentdate
    !                          - description :  current date
    !                          - variablecategory : auxiliary
    !                          - datatype : DATE
    !                          - default : 29/4/2007
    !                          - unit : 
    !                          - uri : some url
    !                          - inputtype : variable
    !            - name: finalLeafNumber
    !                          - description : final leaf number
    !                          - variablecategory : state
    !                          - datatype : DOUBLE
    !                          - min : 0
    !                          - max : 10000
    !                          - default : 8.797582013199484
    !                          - unit : leaf
    !                          - uri : some url
    !                          - inputtype : variable
    !            - name: hasFlagLeafLiguleAppeared
    !                          - description : true if flag leaf has appeared (leafnumber reached finalLeafNumber)
    !                          - variablecategory : state
    !                          - datatype : INT
    !                          - min : 0
    !                          - max : 1
    !                          - default : 1
    !                          - unit : 
    !                          - uri : some url
    !                          - inputtype : variable
    !            - name: phase
    !                          - description :  the name of the phase
    !                          - variablecategory : state
    !                          - datatype : DOUBLE
    !                          - min : 0
    !                          - max : 7
    !                          - default : 1
    !                          - unit : 
    !                          - uri : some url
    !                          - inputtype : variable
        !- outputs:
    !            - name: hasFlagLeafLiguleAppeared
    !                          - description : true if flag leaf has appeared (leafnumber reached finalLeafNumber)
    !                          - variablecategory : state
    !                          - datatype : INT
    !                          - min : 0
    !                          - max : 1
    !                          - unit : 
    !                          - uri : some url
    !            - name: calendarMoments
    !                          - description :  List containing apparition of each stage
    !                          - variablecategory : state
    !                          - datatype : STRINGLIST
    !                          - unit : 
    !            - name: calendarDates
    !                          - description :  List containing  the dates of the wheat developmental phases
    !                          - variablecategory : state
    !                          - datatype : DATELIST
    !                          - unit : 
    !            - name: calendarCumuls
    !                          - description :  list containing for each stage occured its cumulated thermal times
    !                          - variablecategory : state
    !                          - datatype : DOUBLELIST
    !                          - unit : °C d
        IF(phase .GE. 1.0 .AND. phase .LT. 4.0) THEN
            IF(leafNumber .GT. 0.0) THEN
                IF(hasFlagLeafLiguleAppeared .EQ. 0 .AND. (finalLeafNumber .GT. 0.0  &
                        .AND. leafNumber .GE. finalLeafNumber)) THEN
                    hasFlagLeafLiguleAppeared = 1
                    IF(ALL(calendarMoments .NE. 'FlagLeafLiguleJustVisible')) THEN
                        call Add(calendarMoments, 'FlagLeafLiguleJustVisible')
                        call Add(calendarCumuls, cumulTT)
                        call Add(calendarDates, currentdate)
                    END IF
                END IF
            ELSE
                hasFlagLeafLiguleAppeared = 0
            END IF
        END IF
    END SUBROUTINE model_updateleafflag

END MODULE
PROGRAM test
    USE Updateleafflagmod
    REAL:: cumulTT
    REAL:: leafNumber
    CHARACTER(65), ALLOCATABLE, DIMENSION(:):: calendarMoments
    CHARACTER(65), ALLOCATABLE, DIMENSION(:):: calendarDates
    REAL, ALLOCATABLE, DIMENSION(:):: calendarCumuls
    CHARACTER(65):: currentdate
    REAL:: finalLeafNumber
    INTEGER:: hasFlagLeafLiguleAppeared
    REAL:: phase
    print *, "--------test_test_wheat1_UpdateLeafFlag-------"
    hasFlagLeafLiguleAppeared = 0
    phase = 3
    CALL Add(calendarMoments, "Sowing")
    CALL Add(calendarMoments, "Emergence")
    CALL Add(calendarMoments, "EndVernalisation")
    CALL Add(calendarMoments, "MainShootPlus1Tiller")
    CALL Add(calendarMoments, "FloralInitiation")
    CALL Add(calendarMoments, "MainShootPlus2Tiller")
    CALL Add(calendarMoments, "TerminalSpikelet")
    CALL Add(calendarMoments, "PseudoStemErection")
    CALL Add(calendarMoments, "MainShootPlus3Tiller")
    CALL Add(calendarMoments, "1stNodeDetectable")
    CALL Add(calendarMoments, "2ndNodeDetectable")
    CALL Add(calendarMoments, "FlagLeafJustVisible")
    CALL Add(calendarDates, "21/3/2007")
    CALL Add(calendarDates, "27/3/2007")
    CALL Add(calendarDates, "30/3/2007")
    CALL Add(calendarDates, "5/4/2007")
    CALL Add(calendarDates, "9/4/2007")
    CALL Add(calendarDates, "10/4/2007")
    CALL Add(calendarDates, "11/4/2007")
    CALL Add(calendarDates, "12/4/2007")
    CALL Add(calendarDates, "14/4/2007")
    CALL Add(calendarDates, "15/4/2007")
    CALL Add(calendarDates, "19/4/2007")
    CALL Add(calendarDates, "24/4/2007")
    calendarCumuls = [0.0, 112.330110409888, 157.969706915664,  &
            280.570678654207, 354.582294511779, 378.453152853726,  &
            402.042720581446, 424.98704708663, 467.23305195298, 487.544313430698,  &
            560.665248444002, 646.389617338974]
    cumulTT = 741.510096671757
    leafNumber = 8.919453833361189
    currentdate = "29/4/2007"
    finalLeafNumber = 8.797582013199484
    call model_updateleafflag(cumulTT, leafNumber, calendarMoments,  &
            calendarDates, calendarCumuls, currentdate, finalLeafNumber,  &
            hasFlagLeafLiguleAppeared, phase)
    !hasFlagLeafLiguleAppeared: 1
    print *, "hasFlagLeafLiguleAppeared estimated :"
    print *, hasFlagLeafLiguleAppeared
    !calendarMoments: ["Sowing", "Emergence", "EndVernalisation", "MainShootPlus1Tiller", "FloralInitiation", "MainShootPlus2Tiller", "TerminalSpikelet", "PseudoStemErection", "MainShootPlus3Tiller", "1stNodeDetectable", "2ndNodeDetectable", "FlagLeafJustVisible", "FlagLeafLiguleJustVisible"]
    print *, "calendarMoments estimated :"
    Do i_cyml = 1, 13
        print *, calendarMoments(i_cyml);
    END DO
    !calendarDates: ["21/3/2007", "27/3/2007", "30/3/2007", "5/4/2007", "9/4/2007", "10/4/2007", "11/4/2007", "12/4/2007", "14/4/2007", "15/4/2007", "19/4/2007", "24/4/2007", "29/4/2007"]
    print *, "calendarDates estimated :"
    Do i_cyml = 1, 13
        print *, calendarDates(i_cyml);
    END DO
    !calendarCumuls: [0.0, 112.33, 157.97, 280.57, 354.58, 378.45, 402.04, 424.99, 467.23, 487.54, 560.67, 646.39, 741.51]
    print *, "calendarCumuls estimated :"
    Do i_cyml = 1, 13
        print *, calendarCumuls(i_cyml);
    END DO

END PROGRAM
